# Introduction to Functional Programming
## Bee Gibson
## Internal Apps

# What should you get out of this?

- What is the core of FP
- How can it help with problems you've experienced?
- How to use it in projects you're already working on
- Not necessarily hard and complicated
- Neckbeards welcome but not required

# What we will cover

- What makes a language ok for functional programming?
- Immutability
- Putting off side effects
- Referential transparency
- Monads
- Common traps when explaining to others
- How to learn more

# Stop things changing behind my back

# What makes a functional language?

- First class functions - can pass functions to other functions
- other stuff is nice (which python has some of!) but that's the basics to make it workable
- don't need a 'functional' language - I'm a ruby dev and I use mutation so rarely it attracts comments if I do

# Immutability

- Don't change values, use constants and the stack
- Instead of changing, create a new one
- Recursion and accumulators over loops and mutation
- Confusion over passing by value or reference? Gone
- No global mutable state monstrosities without mutable state!

# Code examples

In [1]:
class Item {
    constructor(cost) {
        this.cost = cost;
    };
}

In [2]:
const GST_MULTIPLIER = 1.1

function with_gst(item) {
    // create new Item rather than mutate the argument
    return new Item(item.cost * GST_MULTIPLIER);
};

// Recursion and accumulation over loops and mutation
function add_gst_explicit_recursion(items) {
    [head, ...tail] = items;
    if (tail.length == 0) {
        // base case
        return with_gst(head);
    }
    else {
        // recursive call
        return [with_gst(head)].concat(add_gst_explicit_recursion(tail));
    }
}

add_gst_explicit_recursion([new Item(10), new Item(5)]);

[ Item { cost: 11 }, Item { cost: 5.5 } ]

In [3]:
// We can avoid explicitly disecting the list with map!
function add_gst(items) {
    return items.map(with_gst);
}

add_gst([new Item(10), new Item(5)]);

[ Item { cost: 11 }, Item { cost: 5.5 } ]

In [4]:
// Totalling the shopping cart - reduce the values in to 1

// iterative and mutable - not functional
function mutable_total_cart(items) {
    total = 0;
    for (let i=0; i < items.length; i++) {
        total += items[i].cost;
    };
    return total;
};

mutable_total_cart([new Item(10), new Item(5)]);

15

In [5]:
// explicit recursion

function recursive_total_cart(items) {
    [head, ...tail] = items;
    if (tail.length == 0) {
        // base case
        return head.cost;
    } else {
        // recursive call
        return head.cost + recursive_total_cart(tail);
    };
}

recursive_total_cart([new Item(10), new Item(5)]);

15

In [6]:
// tail call recursion - not useful in JS, but gets memory 
// optimisations in some languages

function tail_call_total_cart(items, total = 0) {
    [head, ...tail] = items;
    if (tail.length == 0) {
        // addition performed in base case
        return total + head.cost;
    } else {
        // last (tail) operation is the recursive call
        // accumulator passed through each call
        return tail_call_total_cart(tail, total + head.cost);
    };
};
    
tail_call_total_cart([new Item(10), new Item(5)]);

15

In [16]:
// implicit recursion

function total_cart(items) {
    // 0 is starting value for accumulator, like the default value of
    // 0 in the previous implementation then run the anonymous function
    // over each value in the items list, using 0 as the initial value
    // for total
    return items.reduce((total, head) => total + head.cost, 0);
};

total_cart([new Item(10), new Item(5)]);

15

# What are side effects?

- when your code effects the outside world
- printing
- writing to disk
- network calls
- it's why you're writing software, what it does

# What do they have to do with FP?

- separate logic from side effects
- push them to the edge of your system
- in the middle make a representation
- make core easier to test and reason about
- no side effects -> simpler tests
- Push hard testing out to the edge, isolated
- not about getting rid of them, about controlling them. Your software won't do much without side effects

In [8]:
function print_add_gst(items, output = console.log) {
    items.map((i) => output(with_gst(i).cost));
};

print_add_gst([new Item(10), new Item(5)]);

11
5.5


In [9]:
// Business logic is seperate, and we can use 
// dependency injection to keep our side effects
// really testable too

function print_cost(items, output=console.log) {
    items.map((i) => output(i.cost));
};

print_cost(add_gst([new Item(10), new Item(5)]));

11
5.5


# Referential transparency

- combination of immutability and no side effects
- call a function again, get the same result and no side effects
- can substitute saved result for function call
- everything a function does represented by the result -> easy reasoning, nothing unexpected

In [10]:
// referentially transparent

function add_gst_cost(item) {
    return with_gst(item)
}

let item = new Item(10)
console.log(add_gst_cost(item).cost)
console.log(add_gst_cost(item).cost)

11
11


In [11]:
// not referentially transparent

function mutable_add_gst_cost(item) {
    item.cost = with_gst(item).cost
    return item
}

item = new Item(10)
console.log(mutable_add_gst_cost(item).cost)
console.log(mutable_add_gst_cost(item).cost)

11
12.100000000000001


# Monads

# Has constructor

# Has flatMap aka bind

# flatten(map(...))

In [12]:
let name_pairs = [["rharded ", "rane"], ["hee ", "hammersky"]]

function capitilise_all(names) {
    return names.map((n) => n.toUpperCase())
}

let transformed_names = name_pairs.map(capitilise_all)

for (let i = 0; i < transformed_names.length; i++) {
    console.log(transformed_names[i])
}

[ 'RHARDED ', 'RANE' ]
[ 'HEE ', 'HAMMERSKY' ]


In [13]:
transformed_names = name_pairs.map(capitilise_all)

for (let i = 0; i < transformed_names.length; i++) {
    for (let j = 0; j < transformed_names[i].length; j++) {
        console.log(transformed_names[i][j])
    }
}

RHARDED 
RANE
HEE 
HAMMERSKY


In [14]:
transformed_names = name_pairs.flatMap(capitilise_all)

for (let i = 0; i < transformed_names.length; i++) {
    console.log(transformed_names[i])
}

RHARDED 
RANE
HEE 
HAMMERSKY


# Generally also have filter

In [15]:
console.log(transformed_names.filter((name) => name.startsWith("R")))

[ 'RHARDED ', 'RANE' ]


# Generally also

# map

# often reduce

# Tail call optimisation

- Makes FP much more efficient by not creating a new stack from for every tail call
- Despite being in ES6 standard it's not in JS implementations except Safari, so can't really use it. Was in V8 for a while, but not anymore.
- Use HO functions - functions that take functions - like map and filter instead
- Also more concise and clearer once you've got your eye in - doesn't have the recursion boilerplate
- They are actually iterative and mutating in JS, but you can reason about it as pure

# Clash with OO?

- Some claim that they clash
- Say that mutation of state is fundamental to OO
- Just making new objects instead of changing them has nothing to do with OO
- Command query separation - related to putting off side effects

# Teaching

- don't get too mathematical, since FP has that reputation
- relate to stuff they know - like monads they have already used
- explaining monads to Venetia
- avoid words like monad until they understand it
- focus on advantages to them, fight the academic and impractical image

# Stop things changing behind our backs

# Questions in Q&A
## https://github.com/beegibson/fp-for-pythonistas
## bee@canva.com